In [21]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/devjaynemorais/modelos_descritivos_curso/main/Aula%2001/dados/dataset_lyrics.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49387 entries, 0 to 49386
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cantorId    49387 non-null  int64 
 1   cantorNome  49387 non-null  object
 2   musicaNome  49387 non-null  object
 3   letra       49376 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [22]:
df.head()

,cantorId,cantorNome,musicaNome,letra
0,0,10000-maniacs,More Than This,I could feel at the time. There was no way of ...
1,0,10000-maniacs,Because The Night,"Take me now, baby, here as I am. Hold me close..."
2,0,10000-maniacs,These Are Days,These are. These are days you'll remember. Nev...
3,0,10000-maniacs,A Campfire Song,"A lie to say, ""O my mountain has coal veins an..."
4,0,10000-maniacs,Everyday Is Like Sunday,Trudging slowly over wet sand. Back to the ben...


In [23]:
df_letras_nulas = df[df['letra'].isnull()]
df_letras_nulas.head()

,cantorId,cantorNome,musicaNome,letra
4943,56,bad-religion,Whisper In Time (traduçâo),NaN
4966,57,badfinger,Baby Blue (traduzida),NaN
4977,57,badfinger,Carry On Till Tomorrow (traduzida),NaN
4986,57,badfinger,Day After Day (traduzida),NaN
12308,141,catch-side,Deixar Partir,NaN


In [24]:
!pip install fuzzywuzzy

In [25]:
# Testando o buscador
texto_busca = "um certo alguém"

In [26]:
# Usando fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def buscar_musica(df, texto_busca, n_resultados=5):
    # Usando a função process.extract do fuzzywuzzy para calcular a similaridade
    resultados = process.extract(texto_busca, df['musicaNome'], scorer=fuzz.token_set_ratio, limit=n_resultados)
    return resultados

resultados = buscar_musica(df, texto_busca)
print("Resultados mais similares:")
for musica, score, id in resultados:
    print(f"Nome da música: {musica}, Similaridade: {score}, ID: {id}")

Resultados mais similares:
Nome da música: Alguém, Similaridade: 100, ID: 11449
Nome da música: Um, Similaridade: 100, ID: 11816
Nome da música: Um certo alguém, Similaridade: 100, ID: 37525
Nome da música: Toda Forma de Amor/um Certo Alguém/o Último Romântico, Similaridade: 100, ID: 37791
Nome da música: Um certo alguém, Similaridade: 100, ID: 37803


In [27]:
# Usando Levenshtein
!pip install python-Levenshtein
from Levenshtein import distance

# Usando Cossine
from scipy.spatial import distance as dist_scipy


In [28]:
df_distancias = df.copy()

In [29]:
df_distancias['Levenshtein'] = df_distancias['musicaNome'].apply(lambda x: distance(texto_busca, x))
# df_distancias['Cossine'] = df_distancias['musicaNome'].apply(lambda x: dist_scipy.cosine(texto_busca, x))




In [30]:
df_distancias.info()

df_Levenshtein = df_distancias.sort_values(by='Levenshtein')
df_Levenshtein.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49387 entries, 0 to 49386
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cantorId     49387 non-null  int64 
 1   cantorNome   49387 non-null  object
 2   musicaNome   49387 non-null  object
 3   letra        49376 non-null  object
 4   Levenshtein  49387 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.9+ MB


,cantorId,cantorNome,musicaNome,letra,Levenshtein
37525,412,lulu-santos,Um certo alguém,Quis evitar teus olhos. Mas não pude reagir. F...,1
37821,412,lulu-santos,Um certo alguém,Quis evitar teus olhos. Mas não pude reagir. F...,1
37803,412,lulu-santos,Um certo alguém,Quis evitar teus olhos. Mas não pude reagir. F...,1
38099,412,lulu-santos,Um certo alguém,Quis evitar teus olhos. Mas não pude reagir. F...,1
46491,493,nxzero,Um Certo Alguém,Quis evitar teus olhos. Mas não pude reagir. F...,3
12433,142,cazuza,Eu Quero Alguém,Eu quero alguém. Na areia da praia. Eu quero a...,6
23776,263,fiuk,Amar Alguém,Nossa história é assim. Eu te vendo e te quere...,7
23762,263,fiuk,Amar Alguém,Nossa história é assim. Eu te vendo e te quere...,7
31579,349,john-people,Sem Medo Algum,Quando menos espera ela pode chegar e mudar. A...,7
23709,263,fiuk,Amar Alguém,Nossa história é assim. Eu te vendo e te quere...,7


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score


# Função para calcular a distância de Jaccard
def jaccard_distancia(str1, str2):
    # Limpar o texto
    # str1, str2 = limpar_texto(str1), limpar_texto(str2)

    # Vetorização
    vectorizer = CountVectorizer(binary=True).fit([str1, str2])
    vec1, vec2 = vectorizer.transform([str1, str2])

    # Cálculo de Jaccard (usando flatten para converter a matriz esparsa)
    return 1 - jaccard_score(vec1.toarray().flatten(), vec2.toarray().flatten())

# Aplicar a função ao DataFrame
df_distancias['Jaccard'] = df_distancias['musicaNome'].apply(lambda x: jaccard_distancia(texto_busca, x))

# Ordenar e exibir os 10 primeiros resultados
df_Jaccard = df_distancias.sort_values(by='Jaccard')
df_Jaccard.head(10)
